In [ ]:
# 1) Prepara datos (año, pivot por país y estándar de tecnologías)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]
for c in year_cols: df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
f = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()
YEAR = [c for c in year_cols if f[c].notna().any()][-1]

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
f["Technology_std"] = f["Technology"].map(tech_map).fillna(f["Technology"])

# Pivot por país (1 punto = país) para el scatter
idx = pd.MultiIndex.from_product([f["Country"].dropna().unique(),
                                  sorted(f["Technology_std"].dropna().unique())],
                                 names=["Country","Technology_std"])
grid = (f.groupby(["Country","Technology_std"])[YEAR].sum(min_count=1)
          .reindex(idx, fill_value=0.0).reset_index())
piv = (grid.pivot(index="Country", columns="Technology_std", values=YEAR)
          .fillna(0.0).reset_index())

for c in ["Solar","Wind","Hydro","Bio","Fossil"]:
    if c not in piv.columns: piv[c] = 0.0
piv["Total"] = piv.drop(columns=["Country"]).sum(axis=1, numeric_only=True)
piv["DominantTech"] = piv[["Solar","Wind","Hydro","Bio","Fossil"]].idxmax(axis=1)

# Para que no se amontonen, quitamos (0,0) solo del SCATTER:
piv_viz = piv[(piv["Solar"]>0) | (piv["Wind"]>0)].copy()
print(f"Año: {YEAR} | países con Solar/Wind>0: {len(piv_viz)}  | filas totales CSV: {len(f)}")

In [1]:
# --- Minimal test cell for the new ScatterBrush (no HTML, generic, quiet) ---
import pandas as pd
import numpy as np
from IPython.display import display

# 1) Use your existing df if present; otherwise make a tiny demo
try:
    _df = df.copy()
except NameError:
    rng = np.random.default_rng(7)
    _df = pd.DataFrame({
        "x": rng.normal(0, 1, 200),
        "y": rng.normal(0, 1, 200),
        "group": rng.choice(["A","B","C"], size=200),
        "size_val": np.clip(rng.normal(6, 2, 200), 1, None),
    })
    _df["Label"] = "pt_" + (np.arange(len(_df))+1).astype(str)
    _df["id"] = np.arange(len(_df)).astype(str)

# 2) Lightweight column picking (generic)
num_cols = list(_df.select_dtypes(include="number").columns)
cat_cols = [c for c in _df.columns if c not in num_cols]

x_col = num_cols[0] if len(num_cols) else _df.columns[0]
y_col = num_cols[1] if len(num_cols) > 1 else (num_cols[0] if num_cols else _df.columns[0])
label_col = "Label" if "Label" in _df.columns else (cat_cols[0] if cat_cols else x_col)
color_col = "group" if "group" in _df.columns else (cat_cols[1] if len(cat_cols) > 1 else None)
size_col  = "size_val" if "size_val" in _df.columns else None
key_col = next((k for k in ["id","ID","key","Key",label_col] if k in _df.columns), label_col)

# 3) Instantiate and display the widget
from Isea.scatter import ScatterBrush
w_scatter = ScatterBrush(
    data=_df,
    x=x_col, y=y_col,
    color=color_col,
    size=size_col,
    label=label_col,
    key=key_col,
    width=1200, height=500,
    panel_position="right",   # or "bottom"
    panel_width=320, panel_height=220,
)
display(w_scatter)

# 4) Capture selection -> pandas DataFrame for linked views (quiet)
def _scatter_sel_observer(change):
    sel = change.get("new") or {}
    rows = sel.get("rows") or []
    scatter_selection_df = pd.DataFrame(rows)
    globals()["scatter_selection_df"] = scatter_selection_df


w_scatter.observe(_scatter_sel_observer, names="selection")
scatter_selection_df = pd.DataFrame([])  # available for downstream cells



ScatterBrush(data=[{'x': 0.0012301533574825742, 'y': -1.2465930582857083, 'group': 'B', 'size_val': 7.05325353…

In [ ]:
# --- Minimal linked view: second scatter shows the selection of w_scatter ---
import pandas as pd
from Isea.scatter import ScatterBrush

# Pull column choices from the first widget so this stays generic
opts = w_scatter.options
x_col = opts.get("x")
y_col = opts.get("y")
label_col = opts.get("label")
color_col = opts.get("color")
size_col  = opts.get("size")
key_col   = opts.get("key")

# Start with empty data; will be filled by the observer below
w_scatter_sel = ScatterBrush(
    data=pd.DataFrame([], columns=[c for c in [x_col, y_col, label_col, color_col, size_col, key_col] if c]),
    x=x_col, y=y_col,
    color=color_col, size=size_col,
    label=label_col, key=key_col,
    squareCells=True,
    width=900, height=450,
    panel_position="right", panel_height=160,
    title="Linked view (selection from first scatter)",
)

display(w_scatter_sel)

# Wire first -> second (live updates)
def _link_selection_to_second(change):
    sel = change.get("new") or {}
    rows = sel.get("rows") or []
    # assign directly; anywidget will sync and the JS will re-render
    w_scatter_sel.data = rows
    # optional: clear selection inside the second chart each update
    w_scatter_sel.selection = {"type": None, "keys": [], "rows": [], "epoch": int(__import__("time").time() * 1000)}

w_scatter.observe(_link_selection_to_second, names="selection")


In [ ]:
scatter_selection_df

In [ ]:
# from Isea import ScatterBrush
# from IPython.display import display
# from ipywidgets import Output
# import pandas as pd

# # sample data (note the actual column names)
# df = pd.DataFrame({
#     "id": [1,2,3,4],
#     "x":  [1,2,3,4],
#     "y":  [10,8,6,9],
#     "group": ["A","B","A","B"]   # <-- categorical for color
# })

# # instantiate with the CORRECT column names
# w_sc = ScatterBrush(
#     df,
#     x="x", y="y",           # <-- must match df
#     color="group",          # optional, only if it exists
#     key="id",               # optional; defaults to a stable id anyway
#     title="My Scatter", x_label="X", y_label="Y",
#     width=720, height=420
# )
# display(w_sc)

# # show selection live below the chart
# out = Output()
# def on_sel(change):
#     sel = change["new"]
#     with out:
#         out.clear_output()
#         if sel.get("type"):
#             rows = sel.get("rows", [])
#             print(f"{sel['type']} selection — {len(rows)} points")
#             display(pd.DataFrame(rows))
#         else:
#             print("Selection cleared")
# w_sc.observe(on_sel, names="selection")
# display(out)


In [ ]:
import pandas as pd
from Isea import ParallelEnergy

df = pd.read_csv("Isea/Energy_clean.csv")

years = [c for c in df.columns if c.startswith("F")]
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])


In [ ]:
w = ParallelEnergy(
    df, years,
    tech_col="Technology_std",
    label_col="Country",
    dims=("Fossil", "Solar", "Hydro", "Wind", "Bio"),
    year_start="F2023",
    width=1100, height=560,
    margin=dict(left=50, right=180, top=80, bottom=36),  # ajusta si quieres
    panel_position="bottom",       # "right" o "bottom"
    panel_width=320, panel_height=220,
    log_axes=False, normalize=False, reorder=True
)
w


In [ ]:
w.show_selection()

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as ipw

out = ipw.Output()
display(out)

def on_sel(change):
    with out:
        clear_output(wait=True)
        df_sel = w.selection_df()
        print(f"point selection — {len(df_sel)} points")
        display(df_sel)

# Escucha cambios del widget
w.observe(on_sel, "selection")


In [ ]:
w2 = w.new_from_selection(
    width=1000, height=520,
    margin=dict(l=48, r=160, t=72, b=32),
    panel_position="bottom",
    panel_height=200
)
w2


In [ ]:
import inspect
from Isea.radial_stacked_bar import RadialStackedBar

# Ver la firma de la función
print(inspect.signature(RadialStackedBar.__init__))

# Ver si year_cols está en los parámetros
params = inspect.signature(RadialStackedBar.__init__).parameters
print("year_cols en parámetros:", "year_cols" in params)

In [ ]:
# ========== PRIMERO: Preparar datos (IGUAL A LA CELDA 1) ==========
import pandas as pd
from Isea import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

# Convertir a numéricas
for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filtrar por indicador y unidad
IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

# IMPORTANTE: Estandarizar tecnologías
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

print(f"✅ Datos preparados: {len(df)} filas")
print(f"✅ Columnas: {df.columns.tolist()}")
print(f"✅ Tecnologías únicas: {df['Technology_std'].unique()}")

# ========== AHORA: Crear RadialStackedBar ==========
radial = RadialStackedBar(
    df=df,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget creado exitosamente!")

In [ ]:
print("Columns:", df.columns.tolist())
print("Has Technology_std:", "Technology_std" in df.columns)
print("Year cols:", year_cols)
print("Sample data:\n", df[["Country", "Technology_std", year_cols[0]]].head())

In [ ]:
# Fuerza recarga
import sys
for mod in list(sys.modules.keys()):
    if 'Isea' in mod:
        del sys.modules[mod]

# Reimporta
from Isea import RadialStackedBar
from IPython.display import display

# Preparar datos (mismo que antes)
import pandas as pd

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

# Crear widget
radial = RadialStackedBar(
    df=df,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget visualizado!")

In [ ]:
from pathlib import Path
from Isea import RadialStackedBar

# Verificar que _esm esté configurado
print("_esm path:", RadialStackedBar._esm)
print("Existe:", Path(RadialStackedBar._esm).exists())

# Si no existe, dinos y crearemos el archivo
print("\nContenido esperado del archivo:")
import inspect
print(inspect.getfile(RadialStackedBar))

In [ ]:
# ========== PREPARACIÓN AGRESIVA DE DATOS ==========
import pandas as pd
from Isea import RadialStackedBar

df = pd.read_csv("Isea/Energy_clean.csv")
year_cols = [c for c in df.columns if c.startswith("F")]

# Convertir a numéricas
for c in year_cols: 
    df[c] = pd.to_numeric(df[c], errors="coerce")

# Filtrar por indicador y unidad
IND, UNIT = "Electricity Installed Capacity", "Megawatt (MW)"
df = df[(df["Indicator"]==IND) & (df["Unit"]==UNIT)].copy()

# Estandarizar tecnologías
tech_map = {
    "Hydropower (excl. Pumped Storage)": "Hydro",
    "Solar energy": "Solar",
    "Wind energy": "Wind",
    "Bioenergy": "Bio",
    "Fossil fuels": "Fossil",
}
df["Technology_std"] = df["Technology"].map(tech_map).fillna(df["Technology"])

# ========== Filtrar solo tecnologías estándar ==========
df_clean = df[
    df["Technology_std"].isin(["Fossil", "Solar", "Wind", "Hydro", "Bio"])
].copy()

# ========== NUEVO: Agrupar por país y tecnología ==========
df_grouped = df_clean.groupby(["Country", "Technology_std"])[year_cols].sum().reset_index()

# ========== NUEVO: Filtrar países con TODAS las tecnologías ==========
# Contar cuántas tecnologías tiene cada país
tech_count = df_grouped.groupby("Country")["Technology_std"].nunique()

# Mantener solo países con LAS 5 TECNOLOGÍAS
required_techs = 5  # Fossil, Solar, Wind, Hydro, Bio
complete_countries = tech_count[tech_count == required_techs].index.tolist()

print(f"📊 Países totales: {df_grouped['Country'].nunique()}")
print(f"✅ Países con 5 tecnologías: {len(complete_countries)}")

# Filtrar
df_grouped = df_grouped[df_grouped["Country"].isin(complete_countries)].copy()

# ========== NUEVO: Eliminar filas donde TODOS los años son 0 ==========
df_grouped["Total"] = df_grouped[year_cols].sum(axis=1)
df_grouped = df_grouped[df_grouped["Total"] > 0].drop(columns=["Total"])

# ========== NUEVO: Filtrar por año más reciente con datos ==========
# Solo mantener países con datos en F2023 (año más reciente)
last_year = "F2023"
df_grouped = df_grouped[df_grouped[last_year] > 0].copy()

print(f"✅ Datos finales después de filtrar: {len(df_grouped)} filas")
print(f"✅ Países únicos: {df_grouped['Country'].nunique()}")
print(f"✅ Tecnologías: {df_grouped['Technology_std'].unique()}")

# ========== Crear RadialStackedBar ==========
radial = RadialStackedBar(
    df=df_grouped,
    group_col="Country",
    category_col="Technology_std",
    year_cols=year_cols,
    year_start="F2023",
    width=900,
    height=900,
    inner_radius=200,
    title="⚡ Capacidad Instalada (MW) - Países Completos",
    custom_colors=["#d73027","#f46d43","#fdae61","#fee08b","#e6f598","#abdda4","#66c2a5","#3288bd"]
)

display(radial)
print("✅ Widget creado con datos limpios!")